## Importing the libraries

In [55]:
!pip install xgboost

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import pickle

## Load and Prepare Data

In [57]:
df = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')
df.info()
df.shape
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [58]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
#As Cabin is full of nan we should drop it, Also name, Ticket numbers and passengerid are not important
df.drop(['Cabin','Ticket','Name','PassengerId'],axis = 1, inplace= True)

In [60]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [61]:
le = LabelEncoder()
df.iloc[:,2] = le.fit_transform(df.iloc[:,2])
df.iloc[:,7] = le.fit_transform(df.iloc[:,7])
df

C:\Users\omidg\AppData\Local\Temp\ipykernel_3616\613177617.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,2] = le.fit_transform(df.iloc[:,2])
C:\Users\omidg\AppData\Local\Temp\ipykernel_3616\613177617.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,7] = le.fit_transform(df.iloc[:,7])


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
888,0,3,0,NaN,1,2,23.4500,2
889,1,1,1,26.0,0,0,30.0000,0


In [62]:
df.dropna(inplace = True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,0,3,0,39.0,0,5,29.1250,1
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
889,1,1,1,26.0,0,0,30.0000,0


In [63]:
X = df.iloc[:,1:]
Y = df.iloc[:,0]

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [65]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_train

In [66]:
def parameter_finder (model, parameters):
    
   
    
    grid = GridSearchCV(model, 
                        param_grid = parameters, 
                        refit = True, 
                        cv = KFold(shuffle = True, random_state = 1), 
                        n_jobs = -1)
    grid_fit = grid.fit(X_train, y_train)
    y_train_pred = grid_fit.predict(X_train)
    y_pred = grid_fit.predict(X_test)
    
    train_score =grid_fit.score(X_train, y_train)
    test_score = grid_fit.score(X_test, y_test)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    
    model_name = str(model).split('(')[0]
    
   
    
    print(f"The best parameters for {model_name} model is: {grid_fit.best_params_}")
    print("--" * 10)
    print(f"(R2 score) in the training set is {train_score:0.2%} for {model_name} model.")
    print(f"(R2 score) in the testing set is {test_score:0.2%} for {model_name} model.")
    print(f"RMSE is {RMSE:,} for {model_name} model.")
    print("--" * 10)
    
    
       
    return train_score, test_score, RMSE

In [12]:
dtr = DecisionTreeClassifier(random_state = 1)
param_dtr = {'min_samples_split': [2, 3, 4, 5],
            'min_samples_leaf': [1, 2, 3]}

dtr_train_score, dtr_test_score, dtr_RMSE = parameter_finder(dtr, param_dtr)

The best parameters for DecisionTreeClassifier model is: {'min_samples_leaf': 3, 'min_samples_split': 2}
--------------------
(R2 score) in the training set is 91.24% for DecisionTreeClassifier model.
(R2 score) in the testing set is 79.72% for DecisionTreeClassifier model.
RMSE is 0.45033010425376047 for DecisionTreeClassifier model.
--------------------


In [13]:
rfr = RandomForestClassifier(random_state = 1, n_jobs = -1)
param_rfr = {'min_samples_split': [2, 3, 4, 5],
            'min_samples_leaf': [1, 2, 3]}

rfr_train_score, rfr_test_score, rfr_RMSE = parameter_finder(rfr, param_rfr)

The best parameters for RandomForestClassifier model is: {'min_samples_leaf': 2, 'min_samples_split': 2}
--------------------
(R2 score) in the training set is 91.59% for RandomForestClassifier model.
(R2 score) in the testing set is 81.12% for RandomForestClassifier model.
RMSE is 0.4345240946267408 for RandomForestClassifier model.
--------------------


In [14]:
xgboost = XGBClassifier(n_jobs = -1)
param_xgboost = {'n_estimators': [100,300],
             'learning_rate': [0.1,0.05],
             'subsample': [0.75],
             'colsample_bytree': [1],
             'max_depth': [3,4,5],
             'gamma': [0]}

xgboost_train_score, xgboost_test_score, xgboost_RMSE = parameter_finder(xgboost, param_xgboost)

The best parameters for XGBClassifier model is: {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.75}
--------------------
(R2 score) in the training set is 90.02% for XGBClassifier model.
(R2 score) in the testing set is 81.82% for XGBClassifier model.
RMSE is 0.4264014327112209 for XGBClassifier model.
--------------------


In [15]:
SvmClassifier = SVC(random_state=1)
param_SvmClassifier = {
    'C' : [0.25,0.5,0.75,1],
    'kernel' : ['linear' , 'rbf'],
    'gamma' : [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}
svm_train_score, svm_test_score, svm_RMSE = parameter_finder(SvmClassifier, param_SvmClassifier)

The best parameters for SVC model is: {'C': 0.25, 'gamma': 0.1, 'kernel': 'linear'}
--------------------
(R2 score) in the training set is 77.58% for SVC model.
(R2 score) in the testing set is 79.72% for SVC model.
RMSE is 0.45033010425376047 for SVC model.
--------------------


In [67]:
XGBC = XGBClassifier(colsample_bytree=1, gamma= 0, learning_rate= 0.1, max_depth=3,n_estimators=100,subsample=0.75,random_state = 1)
XGBC.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=1, ...)

In [68]:
y_hat = XGBC.predict(X_test) 

In [69]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
precision_score(y_test, y_hat) 

0.8

In [70]:
import joblib

In [71]:
joblib.dump(XGBC, 'xgbpipe.joblib') 

['xgbpipe.joblib']

In [72]:
model = joblib.load('xgbpipe.joblib')

## a very important point is that if we want to predict an array like this:

In [73]:
row = np.array([ 3. ,  0. , 28. ,  1. ,  1. , 14.4,  2. ])
row

array([ 3. ,  0. , 28. ,  1. ,  1. , 14.4,  2. ])

# now if we use predict now like this:
## prr = XGBC.predict(row)
### we will encounter an error
#### why? cuz the shape of row is like this:

In [74]:
row.shape

(7,)

In [75]:
row

array([ 3. ,  0. , 28. ,  1. ,  1. , 14.4,  2. ])

# now whats the solution? we should use "RESHAPE" like this:

In [76]:
row = row.reshape(1,7)
row

array([[ 3. ,  0. , 28. ,  1. ,  1. , 14.4,  2. ]])

# Now we can make our prediction(pay attention that row has just one bracket before using RESHAPE while we have double brackets after using it):

In [77]:
prr = XGBC.predict(row)

In [78]:
prr

array([0])

In [79]:
with open("omid.pkl", "wb") as f:
    pickle.dump(XGBC, f) 

In [80]:
import gradio as gr

In [81]:
def make_prediction(Pclass,Sex,Age,SibSp,Parch,Fare,Embarked):
    with open("omid.pkl", "rb") as f:
        clf  = pickle.load(f)
        preds = clf.predict([[Pclass,Sex,Age,SibSp,Parch,Fare,Embarked]])
    if preds == 1:
            return "He will survive"
    return "He will ot survive"

#Create the input component for Gradio since we are expecting 4 inputs

Pclass_input = gr.Number(label = "PClass:1,2,3")
Sex_input = gr.Number(label= "Male:1 , Female:0")
Age_input = gr.Number(label = "Age")
SibSp_input = gr.Number(label = "SibSp")
parch_input = gr.Number(label = "parch")
Fare_input = gr.Number(label = "Fare")
Embarked_input = gr.Number(label = "Embarked")
# We create the output
output = gr.Textbox()


app = gr.Interface(fn = make_prediction, inputs=[Pclass_input,Sex_input, Age_input, SibSp_input, parch_input,Fare_input,Embarked_input], outputs=output)
app.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
